In [1]:
!pip install fastai==2.7.1
!pip install timm>=0.6.2.dev0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 887.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: fastcore
    Found existing installation: fastcore 1.5.6
    Uninstalling fastcore-1.5.6:
      Successfully uninstalled fastcore-1.5.6
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.7
    Uninstalling fastai-2.7.7:
      Successfully uninstalled fastai-2.7.7


In [2]:
from fastai.vision.all import *

In [3]:
train_path = '../input/paddy-disease-classification/train_images'
test_path = '../input/paddy-disease-classification/test_images'

In [4]:
test_files = get_image_files(test_path).sorted()

In [5]:
# Load training Data
dls = ImageDataLoaders.from_folder(train_path, valid_pct=0.1, item_tfms=Resize(640, 480),
        batch_tfms=aug_transforms(size=384, min_scale=0.75), bs=16)

In [6]:
learn = vision_learner(dls, 'convnext_large_in22k', metrics=error_rate, cbs=GradientAccumulation(128)).to_fp16()

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_large_22k_224.pth


In [ ]:
learn.fine_tune(10, 0.005)

epoch,train_loss,valid_loss,error_rate,time
0,1.235391,0.883643,0.300962,12:13


epoch,train_loss,valid_loss,error_rate,time
0,0.723406,0.447691,0.141346,17:01
1,0.563296,0.340524,0.111538,16:54
2,0.358381,0.208112,0.062500,16:45
3,0.297159,0.173389,0.050962,16:46
4,0.234639,0.128661,0.036538,16:45
5,0.181286,0.111735,0.032692,16:54


In [ ]:
preds,_ = learn.tta(dl=dls.test_dl(test_files))

In [ ]:
idxs = preds.argmax(dim=1)
vocab = np.array(dls.vocab)
results = pd.Series(vocab[idxs], name="idxs")

In [ ]:
submit = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')
submit['label'] = results
submit.to_csv('subm.csv', index=False)